In [ ]:
from influxdb_client import InfluxDBClient
from influxdb_client import Point
import requests
from pyowm.owm import OWM
from datetime import datetime, timedelta, time
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from dateutil import tz
import sys
import os
import my
import numpy as np
RED = '\033[91m'
GREEN = '\033[92m'
BLUE = '\033[94m'
YELLOW = '\033[93m'
RESET = '\033[0m'

file_path = 'C:/Users/User/Desktop/UNI/MAGISTRALE/IoT/PROGETTO/conf.py'
directory_path = os.path.dirname(file_path)
sys.path.append(directory_path)
import conf
clear_to_send = False

plants_df = pd.read_excel('../plants.xlsx')
Sens_IDs = plants_df['Sensor ID'].tolist()

client = InfluxDBClient(url=conf.InfluxURL, token=conf.token, org=conf.org)
query_api = client.query_api()
write_api = client.write_api()

query = f'''
from(bucket: "{conf.bucket}")
|> range(start: -6d)
|> filter(fn: (r) => r._measurement == "sens1")
|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''
DF = client.query_api().query_data_frame(org=conf.org, query=query)

per = 12
threshold = 3
city_water = 0
to_water = False
collecting_area = (0.19/2)**2 * np.pi  #cm^2

jreq = requests.put('http://pump1/Water')
req = jreq.json()
my_water = req["Water"]
print(f'Available Water: {BLUE}{my_water} cm^3{RESET}')

df_rain = my.rain_forecast(2)
rain = (df_rain['rain'].mul(df_rain['prob']).sum()) #mm/m^2
print(f'Rain Forecast: {rain / 10} cm/m^2')
collected_water = ((collecting_area**2) * (rain/10))
print(f'Collected water: {BLUE}{collected_water} cm^3{RESET}')
my_water = (my_water + collected_water)#cm^3
print(f'New water amount: {BLUE}{my_water} cm^3{RESET}')

for i, sensor in enumerate(Sens_IDs):
    print(f'Sensor: {GREEN}{sensor}{RESET}')
    min_moisture = plants_df.loc[i, 'Min-Moisture [%]']
    max_moisture = plants_df.loc[i, 'Max-Moisture [%]']
    outdoor = plants_df.loc[i, 'Outdoor']
    roots_h = plants_df.loc[i, 'Roots Depth [m]']
    roots_a = plants_df.loc[i, 'Roots Area [m^2]']
    roots_v = roots_h * roots_a
    last_irrigation = plants_df.loc[i, 'Last Irrigation']

    print(f'MOISTURE LEVELS:\n- min: {RED}{min_moisture}{RESET}%\n- max: {BLUE}{max_moisture}{RESET}%')
    print(f'Roots Volume: {np.round(roots_v, 4)} m^3')
    df_single_plant = DF[DF['Sensor ID'] == sensor]
    df_single_plant = my.DF_for_Prophet(df_single_plant, last_irrigation)
    df_single_plant = df_single_plant.reset_index()
    
    print(f'DataFrame for Prophet implementation:')
    print(df_single_plant.head(4))
    num_rows, num_columns = df_single_plant.shape
    print(f'...\nSize {num_rows} rows x {num_columns} columns')
    
    df = my.clean_df(df_single_plant, 1.5)
    model = Prophet(interval_width=0.95, daily_seasonality = True, weekly_seasonality = False, yearly_seasonality = False)
    model.fit(df)
    future_points = model.make_future_dataframe(periods=per, freq='4H')
    forecast = model.predict(future_points)
    future_val = forecast[forecast['ds'] >= datetime.now()]

    if clear_to_send == True:
        prediction_time = str(datetime.now().date())
        for index, row in future_val.iterrows():
            timestamp = row['ds']
            point = Point("predictions")\
            .tag("Sensor ID", sensor)\
            .tag("Pred_date", prediction_time)\
            .field("pred_moist", row['yhat'])\
            .time(timestamp, 's')
            
            write_api.write(bucket=conf.bucket, org=conf.org, record=point)

    plt.figure(figsize=(10, 6))
    plt.plot(df_single_plant['ds'], df_single_plant['y'], 'bo-', label='Dati Originali')
    plt.plot(df['ds'], df['y'], 'x-', color = 'orange',  label='Dati Processati')
    plt.axhline(y=min_moisture, color='g', linestyle='--', label='Minimium Moisture')
    plt.plot(future_val['ds'], future_val['yhat'], 'r-', label='Previsioni')
    plt.fill_between(future_val['ds'], future_val['yhat_lower'], future_val['yhat_upper'], color='orange', alpha=0.2, label='Intervallo di Confidenza')
    plt.xlabel('Data')
    plt.ylabel('Valore')
    plt.title('Dati Originali e Previsioni')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    violations = future_val[future_val['yhat'] < min_moisture]
    if len(violations) < threshold:
        print(f'The plant monitored by {sensor} does not need to be watered')
        plants_df.loc[i, 'To Be Watered'] = False
        plants_df.loc[i, 'Water [cm^3] - F'] = 0
        plants_df.loc[i, 'City Water [m^3] - F'] = 0
    else:
        last_violation = violations.iloc[threshold-1]
        when = datetime.combine(last_violation['ds'].date(), time(20, 0, 0))
        forecast_moist = last_violation['yhat']
        print(f'Last irrigation occured on: {YELLOW}{last_irrigation}{RESET}')
        print(f'New irrigation scheduled on: {YELLOW}{when}{RESET} (forecasted moisture: {np.round(forecast_moist, 4)}%)')
        
        
        Water = roots_v * (100**2) * (max_moisture - forecast_moist) #cm^3
        print(f'The plant will need: {BLUE}{np.round(Water, 2)}{RESET} cm^3 of water')

        
        if outdoor:
            print(f'The plant is outdoor, so it may catch some rain:')
            rain_on_plant = (0.1 * rain) * roots_a * (roots_a * 100**2)  #cm^3
            plant_water = Water - rain_on_plant
            print(f'In the next days it is going to rain {GREEN}{rain}{RESET} mm/m^2')
            print(f'Water required after the rain: {BLUE}{np.round(plant_water, 4)}{RESET} cm^3')
        else:
            print(f'The water is indoor, so it can not catch any rain')
            plant_water = Water
        
        if(plant_water <= 0):
            to_water = False
        else:
            to_water = True
            
            print(f'I have {np.round(my_water, 4)} cm^3 of water')

            
            if plant_water <= my_water:
                city_water = 0
                my_water = my_water - plant_water 
                print(f'So I can cover the requirements')
            else:
                city_water = plant_water - my_water
                my_water = 0
                print(f'I cannot cover the entire request, I need {np.round(city_water, 4)} cm^3 from the city')
    
            
        plants_df.loc[i, 'To Be Watered'] = to_water
        plants_df.loc[i, 'Scheduled Irrigation'] = when
        plants_df.loc[i, 'Water [cm^3] - F'] = plant_water
        plants_df.loc[i, 'City Water [m^3] - F'] = city_water/(100**3)


df_plants_to_water = plants_df[plants_df['To Be Watered'] == True]
city_water_tot = df_plants_to_water['City Water [m^3] - F'].sum()
print(f'\nTotal Amount Required from the City System: {RED}{city_water_tot}{RESET} m^3')

if clear_to_send == True:
    plants_df.to_excel('../plants.xlsx', index=False)